## A image classification pipeline with PyTorch, model can be a single layer Conv2d.
This example uses torchvision.datasets.FakeData as a placeholder dataset (you can replace it with CIFAR-10, MNIST, or your own).

In [39]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms



In [40]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [41]:
#Define the model
class SingleConvNet(nn.Module):
  def __init__(self, in_channels=1, num_classes=10):
    super().__init__()
    self.conv = nn.Conv2d(in_channels, 16, kernel_size=3, padding=1)
    self.pool = nn.AdaptiveAvgPool2d((1, 1))
    self.classifier = nn.Linear(16, num_classes)

  def forward(self, x):
    x = self.conv(x)          # (B, 16, 28, 28)
    x = torch.relu(x)
    x = self.pool(x)          # (B, 16, 1, 1)
    x = torch.flatten(x, 1)   # (B, 16)
    return self.classifier(x) # (B, 10)

## If we want to add a deeper layer

In [ ]:
 # First convolutional block
    self.conv1 = nn.Conv2d(in_channels, 32, kernel_size=3, padding=1)  # Output: (B, 32, 28, 28)
    self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)                 # Output: (B, 32, 14, 14)

    # Second convolutional block
    self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)           # Output: (B, 64, 14, 14)
    self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)                 # Output: (B, 64, 7, 7)

    # Fully connected layers
    self.fc1 = nn.Linear(64 * 7 * 7, 128)
    self.fc2 = nn.Linear(128, num_classes)

def forward(self, x):
    # Convolutional layers with ReLU and pooling
    x = F.relu(self.conv1(x))
    x = self.pool1(x)

    x = F.relu(self.conv2(x))
    x = self.pool2(x)

    # Flatten
    x = x.view(x.size(0), -1)  # or torch.flatten(x, 1)

    # Fully connected layers
    x = F.relu(self.fc1(x))
    x = self.fc2(x)

    return x

## Dataset and Data loader

In [42]:
#Dataset and data loader

transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
])

train_dataset = datasets.FakeData(size=1000, image_size=(3, 32, 32), num_classes=10, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [43]:
#Use MNIST for testing

transform = transforms.Compose([
    transforms.ToTensor(),
])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

## Training loop

In [44]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SingleConvNet().to(device)

# To use a deeper model with your training loop: run this, then continue with loss function, optimizer, and training loop as before.
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#model = DeepConvNet().to(device)


criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

for epoch in range(5):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader):.4f}")

Epoch 1, Loss: 2.1727
Epoch 2, Loss: 2.0423
Epoch 3, Loss: 2.0267
Epoch 4, Loss: 2.0088
Epoch 5, Loss: 1.9768


## Test evaluation
add a test set in a similar way with model.eval() and no gradients

In [45]:
def evaluate(model, loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = outputs.max(1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
    print(f"Accuracy: {100 * correct / total:.2f}%")

In [46]:
#Add this after training

test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
evaluate(model, test_loader)

Accuracy: 30.22%


In [47]:
# import os
# notebook_path = os.getcwd()
# print(notebook_path)

# path = "/content/drive" # Replace with the path you want to check
# os.path.ismount(path)

## Load PNG dataset and resize

If you have raw .png images (like MNIST digits stored as PNGs), here's a PyTorch-compatible pipeline to:

Load the image from disk

Transform it to tensor

Normalize / resize if needed

Feed it into your SingleConvNet

In [48]:
from PIL import Image
import torch
from torchvision import transforms

# 1. Define transform (for MNIST-like PNGs)
transform = transforms.Compose([
    transforms.Grayscale(),        # ensure it's single-channel
    transforms.Resize((28, 28)),   # resize to 28x28 if needed
    transforms.ToTensor(),         # convert to tensor in [0,1], shape (1, 28, 28)
])

# 2. Load PNG image
#image_path =  '/content/drive/My Drive/Machine learning and DL/CNN_MNP_GUV/PE_no lipids_1hr/0301-8-nolipids-1hr-scale0000.png'
image_path =  '/content/drive/My Drive/Machine learning and DL/CNN_MNP_GUV/number2.png'

img = Image.open(image_path).convert('RGB')  # or 'L' for grayscale

# 3. Apply transform
tensor_img = transform(img)  # shape: (1, 28, 28)

# 4. Add batch dimension and move to device
input_tensor = tensor_img.unsqueeze(0).to("cuda" if torch.cuda.is_available() else "cpu")

# 5. Model inference
model.eval()
with torch.no_grad():
    output = model(input_tensor)
    predicted_label = output.argmax(dim=1).item()

print(f"Predicted Label: {predicted_label}")

Predicted Label: 1


## Notes
.unsqueeze(0) adds the batch dimension: from (1, 28, 28) to (1, 1, 28, 28)

If your model expects RGB (unlikely for MNIST), skip transforms.Grayscale()

If your PNGs are already 28×28 grayscale, just use ToTensor() safely

Next; a batch loader for multiple PNG files in a folder.

In [49]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


After running the cell above and following the instructions to mount your Google Drive, your files will be accessible under `/content/drive/My Drive/`. You'll need to update the `image_path` in the code to reflect the actual location of your image file within your Google Drive.

For example, if you uploaded the image to a folder named `Training data` in your Google Drive, the path might look like this:

In [50]:
image_path = '/content/drive/My Drive/Training data/PE_no lipids_1hr/0301-8-nolipids-1hr-scale0000.png'